In [4]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import h5py
import gdown
from collections import defaultdict
import matplotlib.pyplot as plt
####
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import models 
from tensorflow.keras import layers 
from tensorflow.keras import initializers
from tensorflow.keras.models import load_model
#
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

# 
from collections import defaultdict
from sklearn.preprocessing import StandardScaler

In [ ]:
def reconstr_loss(original_spectra, reconstructed_spectra, latent_dim):
    '''Function to calculate reconstruction loss.

    Inputs:
    - original_spectra (np.array): original spectra.
    - reconstructed_spectra (np.array): reconstruction of the original spectra from the latent representation.
    - latent_dim (integer): size of the latent space.

    Returns:
    - reconstruction loss with added penalty for the latent space size
    '''

    penalty = 0.00003
    penalty2 = 5*0.00003


    mse_loss = mean_squared_error(original_spectra, reconstructed_spectra, squared=True)
    loss_penalized = mse_loss + latent_dim*penalty + penalty2*(latent_dim > 6)

    return(loss_penalized)

In [5]:
data_train = h5py.File( 'train_set.hdf5', 'r')
x_train = np.array( data_train['spectra'] )

data_val = h5py.File( 'val_set.hdf5', 'r')
x_val = np.array( data_val['spectra'] )

data_test = h5py.File('test_set.hdf5', 'r')
x_test = np.array( data_test['spectra'] )

In [6]:
from sklearn.preprocessing import StandardScaler


Scaler = StandardScaler()

x_train_sc = Scaler.fit_transform(x_train)
x_val_sc   = Scaler.transform(x_val)
x_test_sc  = Scaler.transform(x_test)

In [7]:
x_train.shape

(6500, 39974)

In [8]:
def reconstr_loss(original_spectra, reconstructed_spectra, latent_dim):
    '''Function to calculate reconstruction loss.

    Inputs:
    - original_spectra (np.array): original spectra.
    - reconstructed_spectra (np.array): reconstruction of the original spectra from the latent representation.
    - latent_dim (integer): size of the latent space.

    Returns:
    - reconstruction loss with added penalty for the latent space size
    '''

    penalty = 0.00003
    penalty2 = 5*0.00003


    mse_loss = mean_squared_error(original_spectra, reconstructed_spectra, squared=True)
    loss_penalized = mse_loss + latent_dim*penalty + penalty2*(latent_dim > 6)

    return(loss_penalized)


input_dim = x_train.shape[1]
series_input = layers.Input(shape = (x_train.shape[1],1,))


def make_encoder(hidden_nodes_list, activation_functions_list):
    
    

    if len(hidden_nodes_list)!= len(activation_functions_list):
        raise ValueError("length of hidden nodes list should be equal length of activation_functions_list")

    nLayers = len(hidden_nodes_list)
    encoder = models.Sequential()
    encoder.add(series_input)
    for i in range(nLayers): 

        act = activation_functions_list[i]
        n_nodes = hidden_nodes_list[i]
        if i==0:
            encoder.add(layers.Conv1D(n_nodes,16,activation=act, input_shape=[input_dim,1,]))
        else: 
            encoder.add(layers.Conv1D(n_nodes,32,activation=act))

        latent_dim = hidden_nodes_list[-1]
        
    return encoder , latent_dim

def make_decoder(hidden_nodes_list, activation_functions_list, latent_dim):
    latent_input = layers.Input(shape=(1,latent_dim,))
    ## expanding 
    
    if len(hidden_nodes_list)!= len(activation_functions_list):
        raise ValueError("length of hidden nodes list should be equal length of activation_functions_list")
    nLayers = len(hidden_nodes_list)
    decoder = models.Sequential()
    decoder.add(latent_input)
    for i in range(nLayers): 

        act = activation_functions_list[i]
        n_nodes = hidden_nodes_list[i]
        if i==0:
            decoder.add(layers.Conv1DTranspose(n_nodes,32, activation=act, input_shape=(1,latent_dim,)))
        else: 
            decoder.add(layers.Conv1DTranspose(n_nodes, 16,activation=act))
#         decoder.add(layers.Conv1DTranspose(n_nodes, 1,activation='relu'))
    return decoder 

About the convolutional layer

Input shape

3+D tensor with shape: batch_shape + (steps, input_dim)

Output shape

3+D tensor with shape: batch_shape + (new_steps, filters) steps value might have changed due to padding or strides.

In [11]:
ec1_net =models.Sequential()
filters = 30 
delay_size = 7
# input_shape = (train_size,80)

ec1_net.add(layers.Conv1D(filters ,  delay_size,activation='relu', input_shape = (x_train.shape[1],1,) ,padding="same"))
ec1_net.add(layers.MaxPooling1D(pool_size=2))


ec1_net.add(layers.Flatten())
ec1_net.add(layers.Dense(128, activation = 'sigmoid'))
ec1_net.add(layers.Flatten()) 
ec1_net.add(layers.Dense(64,activation='relu'))
ec1_net.add(layers.Flatten()) 
ec1_net.add(layers.Dense(6,activation='relu'))

In [12]:
def make_decoder(hidden_nodes_list, activation_functions_list, latent_dim):
    if len(hidden_nodes_list)!= len(activation_functions_list):
        raise ValueError("length of hidden nodes list should be equal length of activation_functions_list")
    nLayers = len(hidden_nodes_list)
    decoder = models.Sequential()
    for i in range(nLayers): 

        act = activation_functions_list[i]
        n_nodes = hidden_nodes_list[i]
        if i==0:
            decoder.add(layers.Dense(n_nodes, activation=act, input_shape=(latent_dim,)))
        else: 
            decoder.add(layers.Dense(n_nodes,activation=act))
    return decoder

In [14]:
## 
input_dim = x_train.shape[1]


# hidden_nodes = [264,128,6]
hidden_nodes_dec = [128,264,input_dim]

# activation_e1= [ 'linear','relu','linear']#,'tanh','tanh','tanh']

activation_d1= [ 'relu','relu','relu']#,'tanh','tanh','tanh']

earlystopping = EarlyStopping(monitor = 'val_loss', patience = 10)
epochs = 100 
batch_size = 128

# enc_act = ['sigmoid', 'tanh', 'linear','relu']
# for act in enc_act:
# act = 'relu'
# print(f'training for activation {act}')
# activation_e1[1] = act
# encoder,ld = make_encoder(hidden_nodes, activation_e1)
ld = 6
dc1_net = make_decoder(hidden_nodes_dec,activation_d1, latent_dim = ld)
network=models.Sequential()
network.add(ec1_net)
network.add(dc1_net)
network.compile(optimizer='adam', loss='mse', metrics=['mse'])
network.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 6)                 76759094  
_________________________________________________________________
sequential_3 (Sequential)    (None, 39974)             10628062  
Total params: 87,387,156
Trainable params: 87,387,156
Non-trainable params: 0
_________________________________________________________________


In [15]:
ec1_net.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 39974, 30)         240       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 19987, 30)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 599610)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               76750208  
_________________________________________________________________
flatten_4 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
flatten_5 (Flatten)          (None, 64)               

In [16]:
dc1_net.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               896       
_________________________________________________________________
dense_7 (Dense)              (None, 264)               34056     
_________________________________________________________________
dense_8 (Dense)              (None, 39974)             10593110  
Total params: 10,628,062
Trainable params: 10,628,062
Non-trainable params: 0
_________________________________________________________________


In [17]:
x_train_sc.shape

(6500, 39974)

In [18]:

x_train_resh = np.reshape(x_train_sc, (x_train_sc.shape[0], x_train_sc.shape[1], 1))

x_val_resh = np.reshape(x_val_sc, (x_val_sc.shape[0], x_val_sc.shape[1], 1))


history = network.fit(x_train_resh,x_train_resh,
              epochs=epochs,
              batch_size=batch_size,
              callbacks=[earlystopping], # Early stopping
              validation_data=(x_val_resh,x_val_resh))

2021-11-30 09:08:47.003235: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
51/51 [==============================] - 43s 830ms/step - loss: 0.9963 - mse: 0.9963 - val_loss: 1.0252 - val_mse: 1.0252
Epoch 2/100
51/51 [==============================] - 41s 802ms/step - loss: 0.9956 - mse: 0.9956 - val_loss: 1.0251 - val_mse: 1.0251
Epoch 3/100
51/51 [==============================] - 41s 803ms/step - loss: 0.9956 - mse: 0.9956 - val_loss: 1.0250 - val_mse: 1.0250
Epoch 4/100
51/51 [==============================] - 41s 806ms/step - loss: 0.9954 - mse: 0.9954 - val_loss: 1.0247 - val_mse: 1.0247
Epoch 5/100
51/51 [==============================] - 43s 840ms/step - loss: 0.9950 - mse: 0.9950 - val_loss: 1.0244 - val_mse: 1.0244
Epoch 6/100
51/51 [==============================] - 44s 869ms/step - loss: 0.9947 - mse: 0.9947 - val_loss: 1.0242 - val_mse: 1.0242
Epoch 7/100
51/51 [==============================] - 43s 834ms/step - loss: 0.9944 - mse: 0.9944 - val_loss: 1.0238 - val_mse: 1.0238
Epoch 8/100
51/51 [==============================] - 41s 805ms

In [21]:

### train the network 


# weather_input_train = np.expand_dims(weather_input_train, axis=1)

x_test_resh = np.reshape(x_test_sc, (x_test_sc.shape[0], x_test_sc.shape[1], 1))



#     network.save(filepath)

## check reconstruction loss 
test_set_latent_encoding = ec1_net(x_test_resh)
test_set_reconstructions = dc1_net(test_set_latent_encoding)
    # 
x_test_reconstructed = Scaler.inverse_transform(test_set_reconstructions)
x_test_reconstructed = np.reshape(x_test_reconstructed, (x_test_reconstructed.shape[0], x_test_reconstructed.shape[1] ))

rl = reconstr_loss(x_test, x_test_reconstructed, latent_dim = test_set_latent_encoding.shape[1])
print(rl)

0.0060450752819842845
